<a href="https://colab.research.google.com/github/Satwikram/AI-Tutoring/blob/main/NLP/FineTuning%20BERT%20for%20TEXT%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>